Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [3]:
from random import choices

import lab9_lib

In [4]:
fitness = lab9_lib.make_problem(2)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

#print(fitness.__call__(ind))
print(fitness.calls)

10111010111011101011001010011100101100111001111001: 35.80%
01011010101101010010000100100111101111001100110110: 52.00%
10011110110111110100010100100110010111111000110111: 31.40%
10011001111010010001111010001111011011011000110101: 56.00%
11011010010010011000010001111110101111000011110100: 52.00%
01001101100111000100010001100101101010100100110011: 24.00%
00111000010101111010101000101000001100110000000111: 22.20%
00100001001111011110110110000001011011010100010001: 26.20%
01011010011001010101000011001010010011011101001110: 26.00%
00010011000111001000111011110111011011110101100111: 29.40%
10


### PART 1: Just some testing and discover

Techniques to reduce the call of fitness:
1. Caching fitness values to prevent recalculating
2. I don't know, we will see..

Dict for problem instances

In [5]:
problem_instances = {
    1: lab9_lib.make_problem(1),
    2: lab9_lib.make_problem(2),
    5: lab9_lib.make_problem(5),
    10: lab9_lib.make_problem(10)
}

problem_instances

{1: <lab9_lib.make_problem.<locals>.Problem at 0x21648d13ed0>,
 2: <lab9_lib.make_problem.<locals>.Problem at 0x21648d13e90>,
 5: <lab9_lib.make_problem.<locals>.Problem at 0x21648d13f50>,
 10: <lab9_lib.make_problem.<locals>.Problem at 0x21648d13fd0>}

In [6]:
fitness_cache = {}

def evaluate_fitness(genome, fitness_function, cache):
    genome_key = tuple(genome)  # Convert the genome list to a tuple for hashing
    if genome_key not in cache:
        cache[genome_key] = fitness_function(genome)
    return cache[genome_key]

Generate Population

In [7]:
population_size = 500
genome_length = 1000
population = [choices([0, 1], k=genome_length) for _ in range(population_size)]

Tournement Selection

In [8]:
# Fitness Evaluation for the x=10 problem instance
fitness_function = problem_instances[10]

# Calculating fitness for each individual in the population
fitness_values = [evaluate_fitness(individual, fitness_function, fitness_cache) for individual in population]

# Selection - Implementing tournament selection
import random

def tournament_selection(population, fitness_values, tournament_size=10):
    selected_individuals = []
    for _ in range(len(population)):
        # Randomly selecting 'tournament_size' individuals
        tournament = random.sample(list(zip(population, fitness_values)), tournament_size)
        # Selecting the individual with the highest fitness
        winner = max(tournament, key=lambda x: x[1])[0]
        selected_individuals.append(winner)
    return selected_individuals

# Performing tournament selection
# From the population, select the 200 individuals with the highest fitness
selected_population = tournament_selection(population, fitness_values)[:200]

print(len(selected_population))
# Displaying the size of the selected population to verify
for pop in sorted(selected_population, key=lambda x: evaluate_fitness(x, fitness_function, fitness_cache), reverse=True):
    print(f"{fitness_function(pop):.2%}")


200
17.09%
17.09%
17.09%
17.09%
17.09%
16.49%
16.49%
16.49%
15.92%
15.92%
15.91%
15.91%
15.91%
15.91%
15.90%
15.90%
15.90%
15.61%
15.33%
15.33%
15.33%
15.33%
12.00%
12.00%
12.00%
12.00%
12.00%
11.81%
11.81%
11.81%
11.59%
11.59%
11.58%
11.58%
11.58%
11.58%
11.42%
11.42%
11.37%
11.23%
11.23%
11.22%
11.22%
11.20%
11.20%
11.20%
11.19%
11.19%
11.19%
11.18%
11.18%
11.18%
11.18%
11.18%
11.18%
11.18%
11.17%
11.17%
11.17%
11.02%
11.02%
11.02%
11.02%
11.02%
11.02%
11.02%
11.02%
11.02%
11.02%
11.02%
11.02%
11.01%
11.00%
11.00%
11.00%
11.00%
11.00%
11.00%
11.00%
10.99%
10.99%
10.99%
10.99%
10.98%
10.98%
10.98%
10.98%
10.97%
10.97%
10.82%
10.82%
10.82%
10.81%
10.81%
10.81%
10.81%
10.81%
10.81%
10.81%
10.80%
10.80%
10.79%
10.79%
10.79%
10.79%
10.79%
10.78%
10.78%
10.78%
10.78%
10.78%
10.78%
10.78%
10.78%
10.78%
10.61%
10.61%
10.61%
10.61%
10.61%
10.61%
10.61%
10.61%
10.61%
10.61%
10.61%
10.61%
10.61%
10.61%
10.60%
10.60%
10.59%
10.42%
10.42%
10.41%
10.41%
10.41%
10.40%
10.40%
10.40%
10.40%
10.40%
10

Selection, Mutatuion , Crossover (Step 1)

In [9]:
# Implementing Crossover and Mutation
# In this scenario here, what i want to do i to take two parents and produce two offspring and then add them to the new generation.
# I want to do this until the new generation is the same size as the old generation + BETTER FITNESS

def single_point_crossover(parent1, parent2):
    """Perform single point crossover between two parents."""
    if len(parent1) != len(parent2):
        print(f"parents length:{len(parent1)}, {len(parent2)}")
        raise ValueError("Parents must have the same length.")

    crossover_point = random.randint(1, len(parent1) - 1)
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
    return offspring1, offspring2

def bit_flip_mutation(individual, mutation_rate=0.01):
    """Perform bit flip mutation on an individual."""
    mutated_individual = [
        gene if random.random() > mutation_rate else 1 - gene for gene in individual
    ]
    return mutated_individual

def generation(population):
    ''' It will return two generations '''
    # Generating new generation through crossover and mutation
    pop = population.copy()
    new_generation_1 = []
    new_generation_2 = []
    # Creating new generation through crossover and mutation (normal)
    while len(new_generation_1) < (len(pop)//2):
        # Randomly selecting two parents for crossover
        parent1, parent2 = random.sample(pop, 2)
        offspring1, offspring2 = single_point_crossover(parent1, parent2)

        # Applying mutation to offspring
        offspring1 = bit_flip_mutation(offspring1)
        offspring2 = bit_flip_mutation(offspring2)

        new_generation_1.extend([offspring1, offspring2])
    
    # Creating new generation through crossover and mutation (keeping the highest)
    while len(new_generation_2) < (len(pop)//2):
        parent1, parent2 = random.sample(pop, 2)
        offspring1, offspring2 = single_point_crossover(parent1, parent2)

        # Applying mutation to offspring
        offspring1 = bit_flip_mutation(offspring1)
        offspring2 = bit_flip_mutation(offspring2)

        if evaluate_fitness(offspring1, fitness_function, fitness_cache) > evaluate_fitness(parent1, fitness_function, fitness_cache):
            new_generation_2.append(offspring1)
        if evaluate_fitness(offspring2, fitness_function, fitness_cache) > evaluate_fitness(parent2, fitness_function, fitness_cache):
            new_generation_2.append(offspring2)
    
    return new_generation_1, new_generation_2

new_generation_1, new_generation_2 = generation(selected_population)

# Displaying the size of the new generation to verify
print(f"{len(new_generation_1)} {len(new_generation_2)}")


100 100


In [10]:
def combine(gen1 , gen2): 
    ''' It will combine two generations and return len(gen1) best fitnesses '''
    gen1.extend(gen2)
    gen1.sort(key=lambda x: evaluate_fitness(x, fitness_function, fitness_cache), reverse=True)
    return gen1[:len(gen1)//2]

new_generation = combine(new_generation_1, new_generation_2)
print(len(new_generation))
print(f"{evaluate_fitness(new_generation[0], fitness_function, fitness_cache):.2%}")

100
26.41%


In [11]:
# REPEAT THE PROCESS

for _ in range(2):
    print(f" BEFORE: {fitness_function.calls}")
    a , b = generation(new_generation)
    new_generation = combine(a, b)
    print(len(new_generation))
    print(f"{evaluate_fitness(new_generation[0], fitness_function, fitness_cache):.2%}")
    print(f" AFTER: {fitness_function.calls}")

 BEFORE: 1794
50
16.78%
 AFTER: 2348
 BEFORE: 2348
25
17.68%
 AFTER: 2590


### PART 2: Work
The goal of this exercise is to maximize result while minimize number of calls of the fitness, therefore and referring to lab9_lib, what we can do it to maximize the first term of val and minimize the second part

IMP: KEEP CACHING FITNESS

In [12]:
class SolveIt:
    def __init__(self, x, pop_size):
        self.fitness_function = problem_instances[x]
        self.population_size = pop_size
        self.genome_length = 1000
        self.population = [choices([0, 1], k=genome_length, weights=[0.25,0.75]) for _ in range(population_size)]
        self.fitness_cache = {}
    
    def evaluate_fitness(self, genome):
        genome_key = tuple(genome)  # Convert the genome list to a tuple for hashing
        if genome_key not in self.fitness_cache:
            self.fitness_cache[genome_key] = self.fitness_function(genome)
        return self.fitness_cache[genome_key]
    
    def onemax_values(self, anypopulation):
        return [self.fitness_function.onemax(individual) for individual in anypopulation]
    
    def onemax_value(self, elm):
        return self.fitness_function.onemax(elm)
    
    def tournament_selection(self, tournament_size=10):
        selected_individuals = []
        onemax_values = self.onemax_values(self.population)
        for _ in range(len(self.population)):
            tournament = random.sample(list(zip(self.population, onemax_values)), tournament_size)
            winner = max(tournament, key=lambda x: x[1])[0]
            selected_individuals.append(winner)
        return selected_individuals
    
    def calc_smth(self, T):
        fitnesses = []
        for s in range(self.fitness_function.x):
            fitnesses.append(self.onemax_value(T[s:: fitness_function.x]))
        fitnesses.sort(reverse=True)
        return sum(f for f in fitnesses if f == fitnesses[0])
    
    def bit_flip_mutation(self, individual, mutation_rate=0.4):
        mutated_individual = [
            gene if (random.random() > mutation_rate and idx % self.fitness_function.x == 0 ) else 1 - gene for idx, gene in enumerate(individual)
        ]
        return mutated_individual
    
    
    

In [16]:
for i in [1,2,5,10]:
    print(f"Problem Instance: {i}")
    solver = SolveIt(i, 500)
    onemax_values = [solver.fitness_function.onemax(individual) for individual in solver.population]
    selected_population = solver.tournament_selection()[:200]
    print(f"SIZE: {len(selected_population)}")
    for pop in sorted(selected_population, key=lambda x: solver.fitness_function.onemax(x), reverse=True)[:10]:
        print(f"{fitness_function.onemax(pop)} / 1000 are 1's")
    print(f"No of Calls: {solver.fitness_function.calls}")
    j=0
    
    new_generation_1 = []
    new_generation_2 = []
    
    while j < solver.population_size:
        a , b = random.sample(solver.population, 2)
        sp1, sp2 = single_point_crossover(a, b)
        new_generation_1.append(max(sp1,sp2,a,b , key=lambda x: solver.calc_smth(x)))
        j+=1
    
    j=0
    
    while j < solver.population_size:
        a , b = random.sample(solver.population, 2)
        bf1, bf2 = solver.bit_flip_mutation(a, 0.3) , solver.bit_flip_mutation(a, 0.7)
        bf3, bf4 = solver.bit_flip_mutation(b, 0.3) , solver.bit_flip_mutation(b, 0.7)
        new_generation_2.append(max(a,b,bf1,bf2,bf3,bf4 , key=lambda x: solver.calc_smth(x)))
        j+=1
        
    j=0
    k=0
    
    all_gens = []
    
    while k < 5:
        while j < 100:
            individuals = random.sample(new_generation_1, 50)
            inds = random.sample(new_generation_2, 50)
            individuals.extend(inds)
            
            mixture = []
            
            for idx in range(10):
                mutated_individuals = [solver.bit_flip_mutation(invd) for invd in individuals]
                mutated_individuals.sort(key=lambda x: solver.calc_smth(x), reverse=True)
                mutated_individuals = mutated_individuals[:len(mutated_individuals)//2]
                mixture.extend(mutated_individuals)
                #print(f"{len(mutated_individuals)}, {all(len(mutated_individuals[i]) == len(mutated_individuals[i+1]) for i in range(len(mutated_individuals)-1))}")
                crossover_individuals = [single_point_crossover(invd, random.choice(mixture))[0] for invd in individuals]
                crossover_individuals.sort(key=lambda x: solver.calc_smth(x), reverse=True)
                crossover_individuals = crossover_individuals[:len(crossover_individuals)//2]
                mixture.extend(crossover_individuals)

            j+=1
            
        print(f"Generation: {k}")
        
        all_gens.extend(new_generation_1)
        all_gens.extend(new_generation_2)
        all_gens.extend(mixture)
        all_gens.sort(key=lambda x: solver.evaluate_fitness(x), reverse=True)
        all_gens = all_gens[:len(all_gens)//2]
        
        k+=1
        
    print(f"####################################################")
    print(f"FINAL RESULTS FOR PROBLEM INSTANCE {i} TOP 10 FITNESSES")
    for elm in sorted(all_gens, key=lambda x: solver.evaluate_fitness(x), reverse=True)[:10]:
        print(f"{solver.evaluate_fitness(elm):.2%}")
        
    print(f"TOTAL NUMBER OF CALLS: {solver.fitness_function.calls}")
    print(f"####################################################")
    

Problem Instance: 1
SIZE: 200
796 / 1000 are 1's
796 / 1000 are 1's
796 / 1000 are 1's
784 / 1000 are 1's
784 / 1000 are 1's
784 / 1000 are 1's
784 / 1000 are 1's
784 / 1000 are 1's
780 / 1000 are 1's
780 / 1000 are 1's
No of Calls: 0
Generation: 0
SIZE: 1000
Generation: 1
SIZE: 1000
Generation: 2
SIZE: 1000
Generation: 3
SIZE: 1000
Generation: 4
SIZE: 1000
####################################################
FINAL RESULTS FOR PROBLEM INSTANCE 1 TOP 10 FITNESSES
79.60%
79.60%
79.60%
79.60%
79.60%
79.60%
79.60%
79.60%
79.60%
79.60%
TOTAL NUMBER OF CALLS: 1648
####################################################
Problem Instance: 2
SIZE: 200
789 / 1000 are 1's
789 / 1000 are 1's
789 / 1000 are 1's
789 / 1000 are 1's
788 / 1000 are 1's
788 / 1000 are 1's
788 / 1000 are 1's
788 / 1000 are 1's
788 / 1000 are 1's
785 / 1000 are 1's
No of Calls: 0
Generation: 0
SIZE: 1000
Generation: 1
SIZE: 1000
Generation: 2
SIZE: 1000
Generation: 3
SIZE: 1000
Generation: 4
SIZE: 1000
######################

END!